In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import torch
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x

def MFMatrixMulGPU(matrix1, matrix2, batchsize, dev, lower_only=True):
    
    _scores = torch.zeros(matrix1.shape[0], matrix2.shape[0])
    nsplit1 = len(torch.split(matrix1, batchsize))
    nsplit2 = len(torch.split(matrix2, batchsize))

    with torch.no_grad():
        index_i = 0
        for i, batchi in enumerate(torch.split(matrix1,  batchsize)):
            index_j = 0
            for j, batchj in enumerate(torch.split(matrix2,  batchsize)):

                if i>=j and lower_only:
            
                    print(f'({i+1} / {nsplit1}), ({j + 1} / {nsplit2}), ({index_i}-{index_i+batchi.shape[0]}), ({index_j}-{index_j+batchj.shape[0]})')
                    batchi.cuda(dev)
                    batchj.cuda(dev)

                    _batch_scores = abs(torch.matmul(batchi, torch.transpose(torch.conj(batchj), 0, 1)))

                    _scores[index_i:batchi.shape[0] + index_i, index_j:batchj.shape[0] + index_j] = _batch_scores.cpu()
                elif not lower_only:

                    print(f'({i+1} / {nsplit1}), ({j + 1} / {nsplit2}), ({index_i}-{index_i+batchi.shape[0]}), ({index_j}-{index_j+batchj.shape[0]})')
                    batchi.cuda(dev)
                    batchj.cuda(dev)

                    _batch_scores = abs(torch.matmul(batchi, torch.transpose(torch.conj(batchj), 0, 1)))

                    _scores[index_i:batchi.shape[0] + index_i, index_j:batchj.shape[0] + index_j] = _batch_scores.cpu()

                index_j += batchj.shape[0]
            index_i += batchi.shape[0]
            
    return _scores.numpy()



# load data

In [ ]:
os.listdir(os.path.join(DATAPATH, 'dense_template_grid'))

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores'))

In [ ]:
template_banks = os.listdir(os.path.join(DATAPATH, 'dense_template_grid'))

# template-on-template

## use gpu, assume perfect alignment in time so perform simple matrix mul.

In [ ]:
dev = 0

config = {
    'nch': 60, 
    'nslice': 2,
    'samples_per_slice': 8192,
    'batchsize': 100,
    'path': os.path.join(DATAPATH, 'dense_template_grid')
}

#scores = []
for file_name in os.listdir(config['path']):
    #if int(file_name.split('cm')[0][-1]) == 0:
    _path = os.path.join(config['path'], file_name)
    print(file_name)
    _template_data = mf.data.MFDataset(_path)
    _template_metadata = pd.DataFrame(_template_data.metadata)

    _shape = _template_data.data.shape

    # down select the data to have nslice * samples_per_slice samples
    
    _template_data = _template_data.data[:].reshape(_shape[0], config['nch'], _shape[-1] // config['nch'])[:, :, 0:config['nslice']*config['samples_per_slice']].reshape(_shape[0], config['nch'] * config['nslice'] * config['samples_per_slice'])

    #_template_data = torch.from_numpy(np.complex128(_template_data))
    _template_data = torch.from_numpy(_template_data)


    scores = MFMatrixMulGPU(_template_data, _template_data, config['batchsize'], dev)
    
    _name = file_name.split('.h5')[0] + f'_template_scores_nslice{config["nslice"]}'
    
    _save_path = os.path.join(RESULTPATH, 'scores', _name)
    np.save(_save_path, scores)
    


# Plot score images (energy, angle) space

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores'))

In [ ]:
score_list_1 = [
                '211129_sens_est_dense_grid_84.5_0cm_template_scores_nslice2.npy',
                '220107_sens_est_dense_grid_87.0_0cm_template_scores_nslice2.npy',
                '211129_sens_est_dense_grid_89.5_0cm_template_scores_nslice2.npy',
                '220107_sens_est_dense_grid_84.5_4cm_template_scores_nslice2.npy',
                '220107_sens_est_dense_grid_87.0_4cm_template_scores_nslice2.npy',
                '220107_sens_est_dense_grid_89.5_4cm_template_scores_nslice2.npy'
               ]

template_list = [
                '211129_sens_est_dense_grid_84.5_0cm.h5',
                '220107_sens_est_dense_grid_87.0_0cm.h5',
                '211129_sens_est_dense_grid_89.5_0cm.h5',
                '220107_sens_est_dense_grid_84.5_4cm.h5',
                '220107_sens_est_dense_grid_87.0_4cm.h5',
                '220107_sens_est_dense_grid_89.5_4cm.h5',
                ]

signal_list = template_list

In [ ]:
var = 1.38e-23 * 200e6 * 10 * 50 
for i, file_name in enumerate(score_list_1):
    
    scores = np.load(os.path.join(RESULTPATH, 'scores', file_name))
    
    # scores have symmetry for same signal/template
    scores = np.tril(scores) + np.tril(scores).T - np.diag(np.diag(scores))
    

    energy_array = np.linspace(18575, 18580, 101)
    
    central_pitch_angle = float(file_name.split('grid_')[-1].split('_')[0])
    
    angle_array = np.linspace(central_pitch_angle - 0.005, central_pitch_angle + 0.005, 101)
    
    energy_grid, angle_grid = np.meshgrid(energy_array, angle_array)

    _template_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_grid', template_list[i]))
    template_metadata = pd.DataFrame(_template_data.metadata)
    
    
    template_data = _template_data.data[:].reshape(_template_data.shape[0], 60, _template_data.shape[-1] // 60)[:, :, 0:2*8192].reshape(_template_data.shape[0], 2*8192*60)
    
    norm_array = 1 / np.sqrt(var * abs(template_data * template_data.conjugate()).sum(-1))

    _signal_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_grid', signal_list[i]))
    signal_metadata = pd.DataFrame(_signal_data.metadata)

    # choose a random signal
    rng = np.random.default_rng()
    random_index = rng.integers(0, scores.shape[-1], 1)[0]

    signal_energy = signal_metadata.iloc[random_index]['energy']
    signal_angle = signal_metadata.iloc[random_index]['theta_min']

    sorted_scores = np.zeros(energy_grid.size)
    

    for j, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
        
        try:
            #print(i)
            _index = template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0]
            #print(scores[_index, random_index])
            sorted_scores[j] = scores[_index, random_index] * norm_array[_index]
        except:
            #print('Oh no!')
            pass # some of the simulations failed so there will be zero pixels

    plt.figure(figsize=(8, 8))
    plt.imshow(1 * sorted_scores.reshape(energy_grid.shape), aspect='auto', interpolation='none')
    plt.colorbar()
    plt.show()
    



In [ ]:
var = 1.38e-23 * 200e6 * 10 * 50
n_examples = 5

sns.set_theme(context = 'poster', style='ticks')
cmap = sns.color_palette('mako_r', as_cmap=True)

for i, file_name in enumerate(score_list_1):
    
    scores = np.load(os.path.join(RESULTPATH, 'scores', file_name))
    
    # scores have symmetry for same signal/template
    scores = np.tril(scores) + np.tril(scores).T - np.diag(np.diag(scores))

    energy_array = np.linspace(18575, 18580, 101)
    
    central_pitch_angle = float(file_name.split('grid_')[-1].split('_')[0])
    
    radial_position = int(file_name.split('cm')[0].split('_')[-1])
    
    angle_array = np.linspace(central_pitch_angle - 0.005, central_pitch_angle + 0.005, 101)
    
    energy_grid, angle_grid = np.meshgrid(energy_array, angle_array)

    _template_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_grid', template_list[i]))
    template_metadata = pd.DataFrame(_template_data.metadata)
    
    template_data = _template_data.data[:].reshape(_template_data.shape[0], 60, _template_data.shape[-1] // 60)[:, :, 0:2*8192].reshape(_template_data.shape[0], 2*8192*60)
    
    norm_array = 1 / np.sqrt(var * abs(template_data * template_data.conjugate()).sum(-1))

    _signal_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_grid', signal_list[i]))
    signal_metadata = pd.DataFrame(_signal_data.metadata)
    
    for k in range(n_examples):

        # choose a random signal
        rng = np.random.default_rng()
        random_index = rng.integers(0, scores.shape[-1], 1)[0]

        signal_energy = signal_metadata.iloc[random_index]['energy']
        signal_angle = signal_metadata.iloc[random_index]['theta_min']

        sorted_scores = np.zeros(energy_grid.size)


        for j, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):

            try:
                #print(i)
                _index = template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0]
                #print(scores[_index, random_index])
                sorted_scores[j] = scores[_index, random_index] * norm_array[_index]
            except:
                #print('Oh no!')
                pass # some of the simulations failed so there will be zero pixels

        fig = plt.figure(figsize=(13, 8))
        ax = fig.add_subplot(1,1,1)
        img = ax.imshow(
                        1 * sorted_scores.reshape(energy_grid.shape), aspect='auto',
                        interpolation='none',
                        extent=(18575, 18580, central_pitch_angle + 0.005, central_pitch_angle - 0.005),
                       cmap = cmap,
                       )
        cb = fig.colorbar(img, label='Score')
        
        ax.set_xlabel('Energy (eV)')
        ax.set_ylabel('Pitch Angle (deg)')
        ax.set_title('Matched Filter Scores\n T = 10K, Samples = 16384')
        
        
        name = f'220111_mf_score_map_{central_pitch_angle}_{radial_position}cm_example_{k}.png'
        plt.savefig(os.path.join(PATH, 'plots', 'mayfly', name))
        plt.show()
    
